In [6]:
import pandas as pd

df = pd.read_excel('data/股票客户流失.xlsx')

df

,账户资金（元）,最后一次交易距今时间（天）,上月交易佣金（元）,累计交易佣金（元）,本券商使用时长（年）,是否流失
0,22686.5,297,149.25,2029.85,0,0
1,190055.0,42,284.75,3889.50,2,0
2,29733.5,233,269.25,2108.15,0,1
3,185667.5,44,211.50,3840.75,3,0
4,33648.5,213,353.50,2151.65,0,1
...,...,...,...,...,...,...
7038,199145.0,40,424.00,3990.50,1,0
7039,682661.0,1,516.00,9362.90,5,0
7040,51180.5,167,148.00,2346.45,0,0
7041,47594.0,174,372.00,2306.60,0,1


In [7]:
X = df.drop('是否流失', axis=1)

y = df['是否流失']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

df = DecisionTreeClassifier(random_state=0, max_depth=5, min_samples_split=2).fit(X_train, y_train)

df_y_pred = df.predict(X_test)

rfs = RandomForestClassifier(max_depth=5, random_state=0, min_samples_split=2).fit(X_train, y_train)

rfs_y_pred = rfs.predict(X_test)

In [9]:
from sklearn.model_selection import GridSearchCV

df_param_grid = {'max_depth': [5, 6, 7], 'min_samples_split': [2, 3, 4], 'criterion': ['gini', 'entropy']}

rfs_param_grid = {'max_depth': [5, 6, 7], 'min_samples_split': [2, 3, 4], 'criterion': ['gini', 'entropy']}

rfs_cv = GridSearchCV(estimator=rfs, param_grid=rfs_param_grid, cv=5, scoring='accuracy').fit(X_train, y_train)

df_cv = GridSearchCV(estimator=df, param_grid=df_param_grid, cv=5, scoring='accuracy').fit(X_train, y_train)

df_best_model = df_cv.best_estimator_

rfs_best_model = rfs_cv.best_estimator_

In [10]:
from sklearn.metrics import accuracy_score

acc1 = accuracy_score(y_test, df_best_model.predict(X_test))

acc2 = accuracy_score(y_test, rfs_best_model.predict(X_test))

print(acc1, acc2)

0.7853763440860215 0.7879569892473118


In [12]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(random_state=0, penalty='l2', fit_intercept=True).fit(X_train, y_train)

lr_pre = lr.predict(X_test)

acc3 = accuracy_score(y_test, lr_pre)

acc3

0.7918279569892474

In [13]:
lr_pre

array([0, 0, 0, ..., 1, 0, 0], dtype=int64)

In [20]:
import numpy as np


def sigmoid(z):
    return 1 / (1 + np.exp(-z))


def costFunction(X, y, theta):
    A = sigmoid(X.dot(theta))

    first = y * np.log(A)
    second = (1 - y) * np.log(1 - A)

    return -np.sum(first + second) / len(X)


def gradientDescent(X, y, theta, iters, alpha):
    m = len(X)
    costs = np.zeros(iters)

    for i in range(iters):
        A = sigmoid(X @ theta)
        theta = theta - (alpha / m) * X.T.dot(A - y)
        cost = costFunction(X, y, theta)
        costs[i] = cost
    return costs, theta


theta = np.zeros(X.shape[1])

costs, theta = gradientDescent(X, y, theta, 100, 0.001)

pre = sigmoid(X @ theta).astype(int)

pre

D:\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\anaconda3\Lib\site-packages\pandas\core\arraylike.py:396: RuntimeWarning: overflow encountered in exp
  result = getattr(ufunc, method)(*inputs, **kwargs)
D:\anaconda3\Lib\site-packages\pandas\co

0       1
1       1
2       1
3       1
4       1
       ..
7038    1
7039    1
7040    1
7041    1
7042    1
Length: 7043, dtype: int32

In [22]:
acc = np.mean(pre == y)

acc

0.2653698707936959